In [2]:
from sngrok.model import SnMLP
import torch
import functorch

In [21]:
run = torch.load('grokking_s5_40_full_run.pth', map_location=torch.device('cpu'))

In [39]:
run['config']

{'embed_dim': 256, 'model_dim': 128, 'vocab_size': 120, 'tie_lr': False}

In [25]:
run['model']

OrderedDict([('lembed.weight',
              tensor([[ 0.2187, -0.0547, -0.1458,  ..., -0.2085, -0.2089,  0.0953],
                      [-0.0798, -0.0960,  0.1077,  ...,  0.0618,  0.2051, -0.0975],
                      [ 0.2217, -0.0759, -0.2750,  ..., -0.0847,  0.0549,  0.1217],
                      ...,
                      [ 0.2504, -0.1880, -0.1773,  ...,  0.1099,  0.0514, -0.1550],
                      [ 0.1361, -0.2955,  0.2995,  ..., -0.2433, -0.2433, -0.0747],
                      [ 0.0760, -0.1775,  0.2264,  ..., -0.3975, -0.2937, -0.0872]])),
             ('rembed.weight',
              tensor([[ 0.3209, -0.1695,  0.2287,  ..., -0.3052, -0.0731,  0.3047],
                      [-0.3863,  0.1536, -0.2389,  ...,  0.1786, -0.0239, -0.2159],
                      [ 0.0814, -0.1856,  0.0372,  ..., -0.2480, -0.2838,  0.3129],
                      ...,
                      [-0.1716, -0.0908,  0.0995,  ..., -0.1362, -0.3943,  0.3192],
                      [-0.1035,  0.2022, 

In [4]:
cfg = dict(embed_dim = 256, model_dim = 128, vocab_size = 120, tie_lr = False)

In [26]:
model = SnMLP.from_config(cfg)
model.load_state_dict(run['model'])

<All keys matched successfully>

In [27]:
total = 0
for params in model.parameters():
    total += params.numel()
total

142456

(tensor([[  0],
         [  0],
         [  0],
         ...,
         [119],
         [119],
         [119]]),
 tensor([[  0],
         [  1],
         [  2],
         ...,
         [117],
         [118],
         [119]]))

In [28]:
out, cache = model.run_with_cache(torch.tensor(0), torch.tensor(0))

In [63]:
x, y = torch.cartesian_prod(torch.arange(0, 120), torch.arange(0, 120)).hsplit(2)
x, y = x.squeeze(), y.squeeze()
out, cache = model.run_with_cache(x, y)

In [64]:
out.shape

torch.Size([14400, 120])

In [67]:
labels = torch.argmax(out, dim=1)

In [74]:
inverse_idx = torch.where(labels == 0)[0]

In [76]:
list(cache.keys())

['hook_lembed', 'hook_rembed', 'hook_linear', 'hook_unembed']

In [83]:
ident_linear = cache['hook_linear'][inverse_idx]

In [85]:
ident_linear[0]

tensor([ 2.0891e-03, -5.3829e-04,  7.1481e-04,  2.8766e+00, -5.6415e-03,
         2.2936e-04,  4.1506e+00, -4.8906e+00,  5.4126e-05,  4.6513e-05,
        -2.7277e+00,  4.0230e+00, -4.3346e+00,  2.5917e-01, -3.3797e-04,
         6.1628e+00,  8.2757e+00,  3.1471e+00,  4.2812e+00,  9.9865e-02,
         2.2212e+00, -5.2675e+00,  9.0474e-05, -4.4983e+00,  6.3520e+00,
         1.7336e-04, -4.0608e+00,  2.9935e+00, -1.9981e+00,  2.2146e+00,
         4.9082e+00, -3.4551e-04,  1.7498e+00,  1.1121e-02,  2.6951e+00,
         2.6159e+00,  5.0854e+00,  6.6927e+00, -1.8496e+00, -2.5321e+00,
         3.6561e-01, -1.8965e-01,  1.1652e+00,  7.0127e-04, -2.2799e+00,
         7.2127e-05, -2.2385e+00,  2.7018e+00,  8.1320e+00,  5.3831e-02,
         6.2106e-02, -7.9459e+00,  1.9329e+00, -4.3219e-01, -1.9149e+00,
        -1.9712e-05,  2.9418e+00, -4.0672e+00, -2.7144e+00, -4.8627e-05,
        -5.1123e+00,  4.4787e+00, -1.1914e+01,  3.5890e-05,  7.6470e-05,
        -2.4433e+00,  3.0422e-06, -7.9470e+00,  6.1

In [86]:
ident_linear[1]

tensor([-1.6899e-03, -2.0931e-04, -1.3496e-04, -3.3052e+00,  1.2944e-02,
        -1.1349e-04, -4.6190e+00,  5.9948e+00,  6.6291e-04,  1.5360e-03,
         2.3733e+00, -3.0053e+00, -4.3345e+00,  1.6535e-01,  3.3716e-04,
        -2.9817e+00, -3.5128e+00, -4.2073e+00, -4.0156e+00, -4.0383e-01,
        -2.3451e+00,  2.5480e+00, -5.6677e-05,  5.5146e+00, -5.7068e+00,
         2.6489e-04,  3.6479e+00, -2.6034e+00, -1.9967e+00, -2.1119e+00,
        -6.0158e+00,  2.2549e-04, -1.7500e+00, -1.2378e-02, -2.0390e+00,
        -2.3554e+00, -5.6606e+00, -2.8427e+00,  1.8498e+00,  5.9604e+00,
        -2.3456e-01, -2.2291e-01, -5.0821e-01, -2.3419e-03,  1.7473e+00,
        -2.6506e-05,  2.0600e+00, -5.5817e+00, -3.9339e+00,  4.9254e-02,
        -6.2216e-02,  3.3738e+00,  1.9341e+00,  9.9192e-01,  1.9414e+00,
         3.7635e-04, -2.4857e+00,  3.0383e+00,  2.2728e+00,  3.2070e-05,
         5.4531e+00, -5.4917e+00,  1.1914e+01, -9.5631e-05, -3.5162e-04,
         2.1618e+00,  1.1141e-05,  3.8467e+00,  6.1

In [94]:
torch.mean(ident_linear, dim=0)


tensor([ 6.9815e-05, -3.1317e-03, -8.1124e-04, -1.6390e-03,  4.1553e-05,
         1.6015e-03,  2.2237e-03, -1.3779e-03, -3.1226e-03, -1.4115e-02,
         4.9930e-03, -1.2608e-03,  2.7581e-03, -7.2162e-03,  5.5074e-06,
         2.3390e-03, -3.2334e-03, -1.4175e-03, -1.0103e-02, -4.1355e-02,
         2.6702e-02,  7.2012e-04,  6.7687e-06,  1.5777e-03, -3.3673e-03,
        -9.2043e-04,  2.2655e-03, -5.3049e-03,  2.8355e-03,  3.2709e-02,
        -1.5073e-03,  2.5409e-03,  1.6348e-03,  4.1722e-05,  5.3712e-03,
        -1.3143e-02,  4.5205e-03, -2.1973e-03, -2.0434e-04, -1.7507e-03,
        -1.5465e-02,  9.7829e-04, -1.1867e-03,  2.3943e-05,  9.9502e-03,
         7.6730e-07,  2.2443e-02,  6.0860e-04, -9.4678e-04, -2.6680e-02,
        -2.8506e-04,  2.7874e-03, -7.5652e-04,  1.2496e-02, -3.1215e-02,
        -9.1687e-04, -7.1001e-03,  1.4224e-03,  8.5237e-03, -1.1570e-03,
         1.4639e-03,  1.4808e-03, -2.6882e-03, -1.1233e-03, -6.6559e-04,
         1.5240e-02,  1.5179e-06, -2.8379e-03, -3.8

In [93]:
torch.std(ident_linear, dim=0)

tensor([2.9985e-03, 5.1275e-02, 3.5870e-02, 4.7599e+00, 1.7457e-02, 1.8750e-02,
        4.6957e+00, 4.5068e+00, 3.9318e-02, 9.2091e-02, 3.9297e+00, 4.1484e+00,
        6.1830e+00, 3.9298e-01, 2.0755e-04, 4.9001e+00, 6.3149e+00, 4.3358e+00,
        2.9480e+00, 1.5442e+00, 1.5417e+00, 4.1856e+00, 1.3375e-04, 4.1438e+00,
        6.4548e+00, 1.2756e-02, 4.1269e+00, 4.3135e+00, 6.1152e+00, 1.9080e+00,
        4.5206e+00, 3.9274e-02, 4.8403e+00, 2.3718e-02, 4.2260e+00, 1.5766e+00,
        5.7627e+00, 5.1108e+00, 1.8486e+00, 4.5541e+00, 8.8921e-01, 9.4669e-01,
        4.2248e+00, 1.2179e-02, 1.5750e+00, 1.1226e-04, 1.4391e+00, 4.4349e+00,
        6.4613e+00, 9.7514e-01, 8.2074e-02, 6.0683e+00, 5.1649e+00, 3.5963e+00,
        1.9905e+00, 1.2738e-02, 1.9938e+00, 4.1938e+00, 1.1731e+00, 1.6188e-02,
        3.7525e+00, 4.1271e+00, 6.3836e+00, 1.5700e-02, 3.1664e-02, 2.0992e+00,
        5.5472e-05, 6.3189e+00, 1.6499e+00, 1.8810e+00, 3.4323e+00, 5.0496e-03,
        4.0863e+00, 6.5251e+00, 4.1524e-